In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load data
df = pd.read_csv('/mnt/data/application_train.csv')

# Ambil fitur numerik sederhana untuk contoh
features = ['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE']
df = df.dropna(subset=features + ['TARGET'])

X = df[features].values
y = df['TARGET'].values

# Normalisasi
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Ubah label ke -1 dan 1 (SVM binary)
y = y * 2 - 1  # dari (0,1) jadi (-1,1)

# Bagi data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert ke tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Dataset dan DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# SVM model sederhana (Linear)
class LinearSVM(nn.Module):
    def __init__(self, input_dim):
        super(LinearSVM, self).__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        return self.linear(x)

# Hinge loss
def hinge_loss(outputs, labels):
    return torch.mean(torch.clamp(1 - outputs * labels, min=0))

# Training
model = LinearSVM(X.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
epochs = 20

for epoch in range(epochs):
    total_loss = 0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        outputs = model(xb)
        loss = hinge_loss(outputs, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

# Evaluasi
with torch.no_grad():
    preds = model(X_test_tensor)
    predicted_labels = torch.sign(preds)
    acc = (predicted_labels == y_test_tensor).float().mean()
    print(f"Accuracy: {acc:.4f}")


ModuleNotFoundError: No module named 'torch'